In [ ]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
AKSARA_BALI_DIR = pathlib.Path('../data/Aksara-Bali')
AKSARA_SUNDA_DIR = pathlib.Path('../data/Aksara-Sunda')
AKSARA_LAMPUNG_DIR = pathlib.Path('../data/Aksara-Lampung')

In [ ]:
def CreateDatasets(DATA_DIR, type=None):
    data_ds = tf.keras.utils.image_dataset_from_directory(
        DATA_DIR,
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(150, 150),
        seed=1234,
        validation_split=0.2,
        subset='training'
    )
    
    return data_ds

AKSARA_TYPE = 'bali'

if AKSARA_TYPE == 'bali':
    DATA_DIR = AKSARA_BALI_DIR
elif AKSARA_TYPE == 'sunda':
    DATA_DIR = AKSARA_SUNDA_DIR
elif AKSARA_TYPE == 'lampung':
    DATA_DIR = AKSARA_LAMPUNG_DIR
else: raise ValueError('Not on our scripts')

train_ds = CreateDatasets(DATA_DIR, 'training')
valid_ds = CreateDatasets(DATA_DIR, 'validation')

In [ ]:
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)

print(f'Class names: \n{class_names}'
      f'\n\nTotal Class : {NUM_CLASSES}')

In [ ]:
def PlotDatasets(dataset, class_list):
    plt.figure(figsize=(16, 16))
    for images, labels in dataset.take(1):
        for i in range(16):
            plt.subplot(4, 4, i + 1)
            plt.imshow(images[i].numpy().astype('uint8'))
            clas_index = np.argmax(labels[i])
            plt.title(class_list[class_index])
            plt.axis='off'
            
PlotDatasets(train_ds, class_names)

In [ ]:
for image_batch, labels_batch in train_ds:
  print('Batch Image Shape:', image_batch.shape)
  print('Batch Labels Shape:', labels_batch.shape)
  break

In [ ]:
# IF YOU ARE USING TRANSFER LEARNING DON'T RUN THIS

RESCALE = tf.keras.models.Sequential([keras.layers.Rescaling(1./255.)])

train_ds = train_ds.map(lambda x, y: (RESCALE(x), y))
valid_ds = valid_ds.map(lambda x, y: (RESCALE(x), y))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('val_accuracy')>0.999 and logs.get('accuracy')>0.999:
      print("\nReached 99.9% accuracy!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
def save_my_model(model, history, aksara_type=None, type=None):
    accuracy = history.history['accuracy'][-1] * 1000
    val_accuracy = history.history['val_accuracy'][-1] * 1000
    
    model.save(f'../models/model_{type}_{aksara_type}_{int(accuracy)}_{int(val_accuracy)}.h5')

In [ ]:
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(150, 150, 3)
)

base_model.trainable = False

inputs = tf.keras.Input(shape=(150, 150, 3))

model = tf.keras.models.Sequential([
    inputs,
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense((NUM_CLASSES * 15), activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
], name=f'{AKSARA_TYPE}nese script')

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=50,
    verbose=1,
    validation_data=valid_ds,
    callbacks=callbacks
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
save_my_model(model, history, AKSARA_TYPE, 'efficientnet-b0')